In [ ]:
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd
from datetime import datetime as datetime
client = MongoClient("mongodb://localhost:27017")

db = client["lifesnaps"] 
collection = db["fitbit"]

userID = ObjectId("621e2f9167b776a240011ccb")

print("test")



QRY en Projection instellen + query uitvoeren en in dataframe steken

In [ ]:
qry = {
    "id": userID,
    "type": "heart_rate",
  }
projection = {
    "_id": 0
}
rawDocs = list(collection.find(qry,projection))

# Convert to DataFrame
flatDocs = []
for doc in rawDocs:
    data = doc.get("data",{})
    value = data.get("value",{})
    
    if value["confidence"] > 0: # confidence van 0 niet inbegrepen -> te onzeker
        flatDocs.append({
            "dateTime": data["dateTime"],
            "heartrate": value["bpm"]        
    })


df = pd.DataFrame(flatDocs)
df["dateTime"] = pd.to_datetime(df["dateTime"], errors='coerce')
df["heartrate"] = pd.to_numeric(df["heartrate"], errors='coerce')
df.head()


In [ ]:
(df == 0).sum() #Geen missing values

In [ ]:

df["datetime_day"] = df["dateTime"].dt.floor("D")

avgHeartratePerDay = df.groupby("datetime_day")["heartrate"].mean()
print(avgHeartratePerDay)

avgHeartratePerDay.head()

In [ ]:
avgHeartratePerDay.index.name = 'date'
avgHeartratePerDay.to_csv('csv/Heart_Rate.csv',index=True)

In [ ]:
import matplotlib.pyplot as plt

avgHeartratePerDay.plot(title="Average Heart Rate Per Day", ylabel="BPM")
plt.show()

In [ ]:

#voor bepaalde dag

chosenDay = input()
targetDay = pd.to_datetime(chosenDay).date()

# Filter op targetDay
dfDay = df[df["dateTime"].dt.date == targetDay]

# groeperen per groep
dfDay["hour"] = dfDay["dateTime"].dt.hour
avgHeartratePerHour = dfDay.groupby("hour")["heartrate"].mean()

dfChosenDay = df["dateTime"].dt.floor("H")

avgHeartratePerHour.plot(title=f"Average Heart Rate of {targetDay}", ylabel="BPM")
plt.show()


In [ ]:

#grafieken voor verschillende dagen op een rij (hardcoded tho)


chosenDay = "1/1/2022"
targetDay = pd.to_datetime(chosenDay, dayfirst=True).date()  # dayfirst is needed for DD/MM/YYYY
endDay = pd.to_datetime("5/1/2022", dayfirst=True).date()

for day in pd.date_range(start=targetDay, end=endDay, freq='D'):
    # Filter for current day
    dfDay = df[df["dateTime"].dt.date == day.date()].copy()

    if dfDay.empty:
        continue  # skip days with no data

    dfDay["hour"] = dfDay["dateTime"].dt.hour
    avgHeartratePerHour = dfDay.groupby("hour")["heartrate"].mean()

    # Plot
    avgHeartratePerHour.plot(title=f"Average Heart Rate on {day.date()}", ylabel="BPM")
    plt.xlabel("Hour of Day")
    plt.tight_layout()
    plt.show()
